In [124]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

***
No. 1

In [10]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet

In [8]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet

In [125]:
#Green Taxi
df_gr = spark.read.parquet('green_tripdata_2021-02.parquet')

In [126]:
#Yellow Taxi
df_ye = spark.read.parquet('yellow_tripdata_2021-02.parquet')

In [127]:
#Since column for date in green and yellow taxi has differ name, we change one of them into the other
df_ye = df_ye.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime').withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')
df_gr = df_gr.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime').withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [44]:
#Creating tmp view for green taxi and yellow taxi trip
# df_gr.createOrReplaceTempView('green')
# df_ye.createOrReplaceTempView('yellow')

In [128]:
common_colums = []

yellow_columns = set(df_ye.columns)

for col in df_gr.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [131]:
common_colums

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [134]:
df_full = df_gr.select(common_colums).unionAll(df_ye.select(common_colums))

In [135]:
df_full.count()

1436281

In [136]:
df_full.createOrReplaceTempView('full')

In [153]:
# To remove duplicated row if there is any
df_full = spark.sql("""
SELECT
    DISTINCT(*)
FROM
    full
""")

In [138]:
df_full.count()

1436281

In [324]:
# Changing the pickup_datetime column from TimeStamp type to DateTime type
question1 =  df_full.withColumn("pickup_datetime" , to_date(df_full['pickup_datetime'])) \
            .select('pickup_datetime') \
            .filter((df_full['pickup_datetime'] >= '2021-02-15')&(df_full['pickup_datetime'] < '2021-02-16')).count()

In [219]:
question1

45497


1. How many taxi trips were there on February 15?
Ans : 45497
***

***
No. 2

In [253]:
max(df_full['trip_distance'])
max(col('col1')

Column<'max(trip_distance)[values]'>

In [295]:
question2 = df_full.withColumn("pickup_datetime" , to_date(df_full['pickup_datetime']))\
            .select(['pickup_datetime','trip_distance'])\
            .where("pickup_datetime >= '2021-02-01' ")\
            .groupby(F.col('pickup_datetime')).agg(F.max('trip_distance').alias('trip_maxdistance'))



In [292]:
question2.show(100)

+---------------+----------------+
|pickup_datetime|trip_maxdistance|
+---------------+----------------+
|     2021-02-15|           52.89|
|     2021-02-02|           73.24|
|     2021-02-26|        90796.21|
|     2021-02-21|        47327.62|
|     2021-02-05|        91134.16|
|     2021-02-10|         60382.7|
|     2021-02-01|           38.89|
|     2021-02-06|           48.35|
|     2021-02-19|           75.81|
|     2021-03-02|           28.34|
|     2021-02-20|       188054.03|
|     2021-02-08|       186617.92|
|     2021-02-09|        89416.24|
|     2021-02-11|        43174.56|
|     2021-02-17|       140145.44|
|     2021-02-25|        50422.63|
|     2021-03-01|         72499.7|
|     2021-02-27|           91.05|
|     2021-02-24|        90073.44|
|     2021-02-18|        29501.25|
|     2021-02-14|       115928.92|
|     2021-02-12|        66659.27|
|     2021-02-22|           55.87|
|     2021-02-03|       186079.73|
|     2021-02-13|        54381.65|
|     2021-02-23|   


2. Find the longest trip for each day ?
Ans : (Table Above)
***

***
No. 3

In [297]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-02.parquet

In [299]:
df_fhv = spark.read.parquet('fhv_tripdata_2021-02.parquet')

In [306]:
question3 = df_fhv.groupBy("dispatching_base_num").count() \
            .orderBy(F.col('count').desc()).show(5)

+--------------------+-----+
|dispatching_base_num|count|
+--------------------+-----+
|              B00856|35077|
|              B01312|33089|
|              B01145|31114|
|              B02794|30397|
|              B03016|29794|
+--------------------+-----+
only showing top 5 rows




3.  Find Top 5 Most frequent `dispatching_base_num` ?
Ans : (Table Above)
***

***
No. 4

In [318]:
question4 = df_fhv.where("PUlocationID IS NOT NULL AND DOlocationID IS NOT NULL") \
            .groupBy(["PUlocationID",'DOlocationID']) \
            .count() \
            .orderBy(F.col('count').desc()).show(5)

+------------+------------+-----+
|PUlocationID|DOlocationID|count|
+------------+------------+-----+
|       206.0|       206.0| 2374|
|       221.0|       206.0| 2112|
|       129.0|       129.0| 1902|
|         7.0|         7.0| 1829|
|       179.0|       179.0| 1736|
+------------+------------+-----+
only showing top 5 rows




4.  Find Top 5 Most common location pairs (PUlocationID and DOlocationID) ?
Ans : (Table Above)
***

In [351]:
pandasDF = df_gr.toPandas()


In [352]:
pandasDF.to_csv('phasmopobia.csv')

In [350]:
df_gr.coalesce(1).write.parquet('revenue/', mode='overwrite')

Py4JJavaError: An error occurred while calling o1748.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109

In [ ]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [322]:
question1

45497

In [329]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [335]:
# df_gr.coalesce(1).write.option("header","true").format("csv").save("\IYKRA\apachespark\code\csv")

In [353]:
df_gr.write.parquet("P")

Py4JJavaError: An error occurred while calling o1759.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109

In [340]:
from pyspark.sql import SparkSession

BQ_PROJECT_ID = "feisty-outlet-362916"
DATASET_ID = "practicecase_6"
TABLE_NAME = "Question1"

KEY_FILE = "/.google/credentials/google_credentials.json" # When not on GCP
STAGING_BUCKET = "fellowship7"              # Intermediate JSON files
DATASET_LOCATION = "US"                    # Location for dataset creation

# Start session and reference the JVM package via py4j for convienence
session = SparkSession.builder.getOrCreate()
bigquery = session._sc._jvm.com.samelamin.spark.bigquery



In [346]:
spark = SparkSession \
    .builder \
    .master('local') \
    .appName('spark-read-from-bigquery') \
    .config('spark.jars.packages','com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.22.0,com.google.cloud.bigdataoss:gcs-connector:hadoop3-1.9.5,com.google.guava:guava:r05') \
    .config('spark.jars','guava-11.0.1.jar,gcsio-1.9.0-javadoc.jar') \
    .getOrCreate()

In [349]:
df_gr.write.format('bigquery') \
        .mode("overwrite") \
        .option("credentialsFile", "creds.json") \
        .option('table', "feisty-outlet-362916.practicecase_6.df_gr_tested") \
        .option("temporaryGcsBucket","fellowship7") \
        .save()

Py4JJavaError: An error occurred while calling o1743.save.
: java.lang.ClassNotFoundException: 
Failed to find data source: bigquery. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:864)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:256)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:247)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.Dele

In [ ]:
df_gr.write.format('bigquery') \
        .mode("overwrite") \
        .option("credentialsFile", "creds.json") \
        .option('table', table) \
        .option("temporaryGcsBucket",bucket) \
        .save()

In [343]:
 spark = SparkSession \
        .builder \
        .appName(appName) \
        .config(conf=spark_conf) \
        .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.22.0') \
        .config('spark.jars.packages','com.google.cloud.bigdataoss:gcsio:1.5.4') \
        .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar,spark-bigquery-with-dependencies_2.12-0.21.1.jar,spark-bigquery-latest_2.11.jar') \
        .config('spark.jars', 'postgresql-42.2.23.jar,bigquery-connector-hadoop2-latest.jar') \
        .getOrCreate()

NameError: name 'appName' is not defined

In [342]:
# # Saving the data to BigQuery
# df_gr.write.format('bigquery') \
#   .option('table', 'practicecase_6.wordcount_output') \
#   .save()

In [ ]:
# Prepare the bigquery context
bq = bigquery.BigQuerySQLContext(session._wrapped._jsqlContext)
bq.setGcpJsonKeyFile(KEY_FILE)
bq.setBigQueryProjectId(BQ_PROJECT_ID)
bq.setGSProjectId(BQ_PROJECT_ID)
bq.setBigQueryGcsBucket(STAGING_BUCKET)
bq.setBigQueryDatasetLocation(DATASET_LOCATION)

# Extract and Transform a dataframe
# df = session.read.csv(...)

# Load into a table or table partition
bqDF = bigquery.BigQueryDataFrame(df_gr._jdf)
bqDF.saveAsBigQueryTable(
    "{0}:{1}.{2}".format(BQ_PROJECT_ID, DATASET_ID, TABLE_NAME),
    False, # Day paritioned when created
    0,     # Partition expired when created
    bigquery.__getattr__("package$WriteDisposition$").__getattr__("MODULE$").WRITE_EMPTY(),
    bigquery.__getattr__("package$CreateDisposition$").__getattr__("MODULE$").CREATE_IF_NEEDED(),
)